In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from matplotlib import pyplot as plt

In [10]:
# Downloading S&P 500 data for the past 5 years
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0] # Taking the ticker symbols from wikipedia
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-') # Yahoo Finance uses dashes instead of dots
tickers = sp500['Symbol'].unique().tolist() # Get all unique tickers
end_date = pd.to_datetime('2024-03-31') # End date is the last date of FY23-24
start_date = end_date - pd.DateOffset(years=5) # Taking 5 years of data
df = yf.download(tickers, start=start_date, end=end_date) # Downloading data

[*********************100%%**********************]  503 of 503 completed


8 Failed downloads:
['PSX', 'SWKS', 'XYL', 'IDXX', 'WRK', 'CZR']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2019-03-31 00:00:00 -> 2024-03-31 00:00:00)')
['SOLV', 'GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1554004800, endDate = 1711857600")


In [11]:
adj_close = df['Adj Close'] # Taking the adjusted close prices
adj_close = adj_close.dropna(axis=1) # Dropping columns with all NaN values
adj_close.to_csv('sp500_adj_close.csv') # Saving the data to a csv file
print("Shape of the data in (Days, Companies):", adj_close.shape)

Shape of the data in (Days, Companies): (1258, 486)
